In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime

In [2]:
# --- Load Excel File ---
excel_path = r'Z:\Power BI\TLC\Data & DFR\ADT_27.06.xlsx' 
sheet_name = 'Sheet1'
df = pd.read_excel(excel_path, sheet_name=sheet_name)
from datetime import timedelta

In [3]:
# --- Define Date Range ---
today = datetime.today()
start_date = (today - timedelta(days=30)).strftime('%Y-%m-%d')
end_date = today.strftime('%Y-%m-%d')
date_column = 'Call date'

In [4]:
# Convert to datetime
if 'df' in locals():
	df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
else:
	print("DataFrame 'df' is not defined. Please run the cell where 'df' is created.")

In [5]:
# Filter records for the selected date range
filtered_df = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]

In [6]:
# ---  MySQL Connection Setup ---
db_config = {
    'host': '172.27.100.150',         
    'user': 'clink',     
    'password': 'Cl1nk#09876',
    'database': 'TLC_HF', 
    'port': 3306
}
table_name = 'ADT_Automation_copy'
engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_config['database']}"
)

In [7]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cryptography

In [9]:
# --- Select Data from MySQL ---
with engine.connect() as conn:
    result = conn.execute(text(f"""
        SELECT COUNT(*) FROM {table_name} WHERE `{date_column}` BETWEEN :start_date AND :end_date
    """), {'start_date': start_date, 'end_date': end_date})
    count = result.scalar()
    print(f"Rows matching condition: {count}")


Rows matching condition: 976


In [10]:
from sqlalchemy import Table, MetaData, delete
metadata = MetaData()
table = Table(table_name, metadata, autoload_with=engine)
stmt = delete(table).where(table.c[date_column].between(start_date, end_date))
with engine.begin() as conn:
    result = conn.execute(stmt)
    print(f"Deleted rows: {result.rowcount}")

Deleted rows: 976


In [13]:
# --- Select Data from MySQL --- Aftr deleting the rows
with engine.connect() as conn:
    result = conn.execute(text(f"""
        SELECT COUNT(*) FROM {table_name} WHERE `{date_column}` BETWEEN :start_date AND :end_date
    """), {'start_date': start_date, 'end_date': end_date})
    count = result.scalar()
    print(f"Rows matching condition: {count}")


Rows matching condition: 0


In [ ]:
# - Create a new DataFrame with the filtered data & Export to CSV
csv_path = r'Z:\Power BI\TLC\Data & DFR\filtered_data.csv'
filtered_df.to_csv(csv_path, index=False)
print(f"Filtered data saved to {csv_path}")

In [14]:
from sqlalchemy import create_engine

engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_config['database']}"
)
import pandas as pd


In [15]:
# Load the mapping CSV
mapping_df = pd.read_csv(r'Z:\Power BI\TLC\column_mapping.csv')
mapping_dict = dict(zip(mapping_df['csv_column'], mapping_df['mysql_column']))

In [16]:
# Load your data CSV
data_df = pd.read_csv(r'Z:\Power BI\TLC\Data & DFR\filtered_data.csv')

In [17]:
# Rename columns using the mapping
data_df = data_df.rename(columns=mapping_dict)

In [18]:
mysql_columns = mapping_df['mysql_column'].tolist()
existing_columns = [col for col in mysql_columns if col in data_df.columns]
data_df = data_df[existing_columns]

In [19]:
from sqlalchemy import create_engine

engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}@"
    f"{db_config['host']}:{db_config['port']}/{db_config['database']}"
)

In [20]:
data_df.to_sql(table_name, con=engine, if_exists='append', index=False)
print(f"Inserted {len(data_df)} rows into {table_name}.")
from sqlalchemy import text

Inserted 976 rows into ADT_Automation_copy.


In [21]:
# Define the unique_id column name
unique_id = 'Unique id'

try:
    with engine.connect() as conn:
        # Get total row count in the table
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
        count = result.scalar()
        
        # Get unique ids for the inserted date range
        result = conn.execute(text(f"""
            SELECT `{unique_id}` FROM {table_name}
            WHERE `{date_column}` BETWEEN :start_date AND :end_date
        """), {'start_date': start_date, 'end_date': end_date})
        unique_ids = [row[0] for row in result.fetchall()]
        
        # Print summary info
        print("✅Database updated closed successfully!")
        
        print(
            f"Total rows in table: {count} | "
            f"Inserted {len(data_df)} rows into {table_name}. | "
            f"start_date: {start_date}, end_date: {end_date}\n"
            f"Unique IDs in date range:"
        )
        # Print each unique id on a new line
        for uid in unique_ids:
            print(uid)
        print("Query execution completed. Closing the database connection...")

    print("✅Database connection closed successfully.")
except Exception as e:
    print("❌ Code execution failed.")
    print(f"Error details: {e}")


✅Database updated closed successfully!
Total rows in table: 3103 | Inserted 976 rows into ADT_Automation_copy. | start_date: 2025-06-01, end_date: 2025-07-01
Unique IDs in date range:
683f13b8d0687
683f157fe4bba
683f191525985
683f22bb3113a
683f2a88ac95a
683f3a2bbd4a5
683f3bf3c495a
683f58ac3eebe
683f627c05a48
683f6e3d73eaa
68405b8400160
6840637d28f14
684083d47c52a
68408dd2da5b5
68408ebeafdfe
684096f0434c8
6840a05a60b74
68408c108654f
6840a4d3db3da
6841abd056a14
6841bd88efec5
6841c69d0eba5
6841d1b6f3db0
6841d5914dc29
6841e3a8ca5cd
6841dcc654b31
6841f4150ac69
684318d414bb8
684337061a794
684367a928a70
68436cc299911
68443b3c436a6
68444973147ce
68448dce260a6
6846fcd9ef172
`6847029e95548
684707266576d
684710f6d7910
68499ace329b1
6849a500b6e72
6849bf4fa7008
6849decc304a3
684b18cc2acde
684b2f03153cf
684cba4f21ddc
68506a2c674dc
685091fa4507a
68517ee2a7c52
6851942a11616
6851d16f29499
6851d5210bfc9
6851eb694b8fc
6852e5d151124
68532e72f20eb
6856b7d95fea1
68596feb71298
6859800e6569e
`6859763869860
68